# <center> Introduction to Spark In-memmory Computing via Python PySpark </center>

## 1. Getting Started

Spark stores data in memory. This memory space is represented by variable **sc** (SparkContext). 

In [1]:
sc

The Spark cluster created through this notebook's kernel is running on Cypress' Hadoop infrastructure. Therefore, the easiest storage location for Spark is Cypress' HDFS. 

In [ ]:
!hdfs dfs -mkdir intro-to-spark

In [ ]:
!hdfs dfs -put /home/lngo/intro-to-spark/airlines/ intro-to-spark/
!hdfs dfs -put /home/lngo/intro-to-spark/movielens/ intro-to-spark/
!hdfs dfs -put /home/lngo/intro-to-spark/text/ intro-to-spark/

In [ ]:
!hdfs dfs -ls intro-to-hadoop/text

In [5]:
textFile = sc.textFile("intro-to-spark/text/gutenberg-shakespeare.txt")

In [6]:
print (textFile)

intro-to-spark/text/gutenberg-shakespeare.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:-2


## 2. What does Spark do with my data?

**Storage Level:**
- Does RDD use disk?
- Does RDD use memory?
- Does RDD use off-heap memory?
- Should an RDD be serialized (while persisting)?
- How many replicas (default: 1) to use (can only be less than 40)?

In [4]:
textFile.getStorageLevel()

StorageLevel(False, False, False, False, 1)

In [7]:
textFile.getNumPartitions()

2

In [8]:
textFile.cache()

intro-to-spark/text/gutenberg-shakespeare.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:-2

In [9]:
textFile.getStorageLevel()

StorageLevel(False, True, False, False, 1)

- By default, each transformed RDD may be recomputed each time you run an action on it.
- It is also possible to *persist* RDD in memory using *persist()* or *cache()*
    - *persist()* allows you to specify level of storage for RDD
    - *cache()* only persists RDD in memory
    - To retire RDD from memory, *unpersist()* is called

## 3. WordCount

Data operations in Spark are categorized into two groups, *transformation* and *action*. 
- A *transformation* creates new dataset from existing data. Examples of *transformation* include map, filter, reduceByKey, and sort. 
- An *action* returns a value to the driver program (aka memory space of this notebook) after running a computation on the data set. Examples of *action* include count, collect, reduce, and save. 

"All transformations in Spark are lazy, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result to be returned to the driver program." -- Spark Documentation

#### RDD Operations in Spark

**Transformations: **

- *map*(f: T -> U) : RDD[T] -> RDD[U]
- *filter*(f: T -> Bool) : RDD[T] -> RDD[T]
- *flatMap*(f: T -> Seq[U]) : RDD[T] -> RDD[U]
- *sample*(*fraction*: Float) : RDD[T] -> RDD[T] (deterministic sampling)
- *groupByKey*() : RDD[(K,V)] -> RDD[(K, Seq[V])]
- *reduceByKey*(f: (V,V) -> V) : RDD[(K,V)] -> RDD[(K,V)]
- *union*() : (RDD[T], RDD[T]) -> RDD[T]
- *join*() : (RDD[(K,V)], RDD[(K,W)]) -> RDD[(K,(V,W))]
- *cogroup*() : (RDD[(K,V)], RDD[(K,W)] -> RDD[(K, (Seq[V],Seq[W]))]
- *crossProduct*() : (RDD[T], RDD[U]) -> RDD[(T,U)]
- *mapValues*(f: V -> W) : RDD[(K,V)] -> RDD[(K,W)] (preserves partitioning)
- *sort*(c: Comparator[K]) :  RDD[(K,V)] -> RDD[(K,V)]
- *partitionBy*(p: Partitioner[K]) : RDD[(K,V)] -> RDD[(K,V)]

**Actions:**

- *count*() : RDD[T] -> Long
- *collect*() : RDD[T] -> Seq[T]
- *reduce*(f: (T,T) -> T) : RDD[T] -> T
- *lookup*(k : K) : RDD[(K,V)] -> Seq[V] (on hash/range partitionied RDDs)
- *save*(path: String) : Outputs RDD to a storage system 

In [ ]:
textFile = sc.textFile("intro-to-spark/text/gutenberg-shakespeare.txt")

In [ ]:
textFile

In [ ]:
%%time
textFile.count()

In [ ]:
wordcount = textFile.flatMap(lambda line: line.split(" ")) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)

In [ ]:
wordcount

In [ ]:
!hdfs dfs -rm -r intro-to-spark/output-wordcount-01
wordcount.saveAsTextFile("intro-to-spark/output-wordcount-01")

In [ ]:
!hdfs dfs -ls intro-to-spark/output-wordcount-01

In [ ]:
!hdfs dfs -cat intro-to-spark/output-wordcount-01/part-00000 \
    2>/dev/null | head -n 20

**Step-by-step actions:**

In [ ]:
!hdfs dfs -cat intro-to-spark/text/gutenberg-shakespeare.txt \
    2>/dev/null | head -n 10

In [ ]:
wordcount_step_01 = textFile.flatMap(lambda line: line.split(" "))

In [ ]:
wordcount_step_01

In [ ]:
wordcount_step_01.take(20)

In [ ]:
wordcount_step_02 = wordcount_step_01.map(lambda word: (word, 1))

In [ ]:
wordcount_step_02.take(20)

In [ ]:
wordcount_step_03 = wordcount_step_02.reduceByKey(lambda a, b: a + b)

In [ ]:
wordcount_step_03.take(20)

## 4. Movie Ratings

An independent movie company is looking to invest in a new movie project. With limited finance, the company wants to 
analyze the reaction of audiences, particularly toward various movie genres, in order to identify beneficial 
movie project to focus on. The company relies on data collected from a publicly available recommendation service 
by [MovieLens](http://dl.acm.org/citation.cfm?id=2827872). This 
[dataset](http://files.grouplens.org/datasets/movielens/ml-10m-README.html) contains **22884377** ratings and **586994**
 tag applications across **34208** movies. These data were created by **247753** users between January 09, 1995 and January 29, 2016. This dataset was generated on January 29, 2016. 

From this dataset, several analyses are possible, include the followings:
1.   Find movies which have the highest average ratings over the years and identify the corresponding genre.
2.   Find genres which have the highest average ratings over the years.
3.   Find users who rate movies most frequently in order to contact them for in-depth marketing analysis.

These types of analyses, which are somewhat ambiguous, demand the ability to quickly process large amount of data in 
elatively short amount of time for decision support purposes. In these situations, the sizes of the data typically 
make analysis done on a single machine impossible and analysis done using a remote storage system impractical. For 
remainder of the lessons, we will learn how HDFS provides the basis to store massive amount of data and to enable 
the programming approach to analyze these data.

In [ ]:
!hdfs dfs -ls -h intro-to-spark/movielens

In [ ]:
!hdfs dfs -cat intro-to-hadoop/movielens/README.txt

In [ ]:
!hdfs dfs -cat intro-to-hadoop/movielens/links.csv \
    2>/dev/null | head -n 5

In [ ]:
!hdfs dfs -cat intro-to-hadoop/movielens/movies.csv \
    2>/dev/null | head -n 5

In [ ]:
!hdfs dfs -cat intro-to-hadoop/movielens/ratings.csv \
    2>/dev/null | head -n 5

In [ ]:
!hdfs dfs -cat intro-to-hadoop/movielens/tags.csv \
    2>/dev/null | head -n 5

In [2]:
ratings = sc.textFile("intro-to-spark/movielens/ratings.csv")

In [3]:
ratings.cache()

intro-to-spark/movielens/ratings.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [4]:
%%time
ratings.count()

CPU times: user 26.1 ms, sys: 12.3 ms, total: 38.3 ms
Wall time: 55.2 s


22884378

In [5]:
%%time
ratings.count()

CPU times: user 19.4 ms, sys: 7.16 ms, total: 26.5 ms
Wall time: 41.8 s


22884378

In [6]:
%%time
ratings.count()

CPU times: user 15.5 ms, sys: 5.16 ms, total: 20.6 ms
Wall time: 32.9 s


22884378

### 4.1 Find movies which have the highest average ratings over the years and identify the corresponding genre

- Find the average ratings of all movies over the years
- Identify the corresponding genres for each movie

In [7]:
ratings.take(5)

[u'userId,movieId,rating,timestamp',
 u'1,169,2.5,1204927694',
 u'1,2471,3.0,1204927438',
 u'1,48516,5.0,1204927435',
 u'2,2571,3.5,1436165433']

In [8]:
ratingHeader = ratings.first() #extract header
print(ratingHeader)

userId,movieId,rating,timestamp


In [9]:
ratingsOnly = ratings.filter(lambda x:x != ratingHeader)

In [10]:
ratingsOnly.take(5)

[u'1,169,2.5,1204927694',
 u'1,2471,3.0,1204927438',
 u'1,48516,5.0,1204927435',
 u'2,2571,3.5,1436165433',
 u'2,109487,4.0,1436165496']

In [11]:
movieRatings = ratingsOnly.map(lambda line: (line.split(",")[1], float(line.split(",")[2])))

In [12]:
movieRatings.take(5)

[(u'169', 2.5),
 (u'2471', 3.0),
 (u'48516', 5.0),
 (u'2571', 3.5),
 (u'109487', 4.0)]

**Possible approaches in aggregating data:** 
- groupByKey and mapValues
- reduceByKey and countByKey

**groupByKey and mapValues**

In [13]:
groupByKeyRatings = movieRatings.groupByKey()

groupByKeyRatings.take(5)

[(u'73399', <pyspark.resultiterable.ResultIterable at 0x7f43e43e3490>),
 (u'110555', <pyspark.resultiterable.ResultIterable at 0x7f43e43ec9d0>),
 (u'73462', <pyspark.resultiterable.ResultIterable at 0x7f43e43ec950>),
 (u'145208', <pyspark.resultiterable.ResultIterable at 0x7f43e43ec110>),
 (u'89373', <pyspark.resultiterable.ResultIterable at 0x7f43e43ec810>)]

In [14]:
mapValuesToListRatings = groupByKeyRatings.mapValues(list)

mapValuesToListRatings.take(5)

[(u'73399', [3.5, 3.5, 2.5, 3.5, 2.0, 3.5, 2.5, 3.0]),
 (u'110555', [4.0]),
 (u'73462',
  [1.5,
   0.5,
   3.0,
   1.0,
   3.0,
   2.5,
   5.0,
   0.5,
   2.0,
   2.0,
   3.5,
   2.5,
   5.0,
   4.0,
   3.0,
   1.5,
   2.0,
   2.5,
   0.5,
   0.5,
   1.5,
   0.5,
   3.0,
   3.0,
   4.5,
   3.5,
   4.0,
   0.5,
   3.0,
   0.5,
   0.5,
   3.5,
   3.0,
   1.0,
   2.5,
   5.0,
   1.5,
   3.0,
   4.0,
   1.0,
   1.0,
   0.5,
   3.0,
   0.5,
   3.0,
   3.0,
   0.5,
   3.5,
   2.5,
   1.5,
   0.5,
   2.0,
   0.5,
   3.0]),
 (u'145208', [2.0]),
 (u'89373', [2.0, 4.0, 3.5, 4.5, 1.5, 2.0, 4.5, 3.0, 3.0, 1.0])]

In [15]:
avgRatings01 = mapValuesToListRatings.mapValues(lambda V: sum(V) / float(len(V)))

avgRatings01.take(5)

[(u'73399', 3.0),
 (u'110555', 4.0),
 (u'73462', 2.2222222222222223),
 (u'145208', 2.0),
 (u'89373', 2.9)]

Is this correct?

In [16]:
(3.5 + 3.5 + 2.5 + 3.5 + 2.0 + 3.5 + 2.5 + 3.0) / 8

3.0

**reduceByKey and countByKey**

In [ ]:
countsByKey = movieRatings.countByKey()

countsByKey

In [18]:
def sumValues(x,y):
    return (x + y)

sumRatings = movieRatings.reduceByKey(sumValues)

sumRatings.take(5)

[(u'73399', 24.0),
 (u'110555', 4.0),
 (u'73462', 120.0),
 (u'145208', 2.0),
 (u'89373', 29.0)]

In [ ]:
import operator

sumRatings = movieRatings.reduceByKey(operator.add)
sumRatings.take(5)

In [19]:
avgRatings02 = sumRatings.map(lambda x: (x[0], x[1] / countsByKey.get(x[0])))

avgRatings02.take(5)

[(u'73399', 3.0),
 (u'110555', 4.0),
 (u'73462', 2.2222222222222223),
 (u'145208', 2.0),
 (u'89373', 2.9)]

How do we augment movie ratings data with title informations?

In [20]:
movies = sc.textFile("intro-to-spark/movielens/movies.csv")

In [21]:
movieHeader = movies.first() #extract header
print(movieHeader)

movieId,title,genres


In [22]:
movies = movies.filter(lambda x:x != movieHeader)

movies.take(5)

[u'1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 u'2,Jumanji (1995),Adventure|Children|Fantasy',
 u'3,Grumpier Old Men (1995),Comedy|Romance',
 u'4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 u'5,Father of the Bride Part II (1995),Comedy']

In [23]:
movieInfo = movies.map(lambda line: (line.split(",")[0], (line.split(",")[1], line.split(",")[2])))

movieInfo.take(5)

[(u'1', (u'Toy Story (1995)', u'Adventure|Animation|Children|Comedy|Fantasy')),
 (u'2', (u'Jumanji (1995)', u'Adventure|Children|Fantasy')),
 (u'3', (u'Grumpier Old Men (1995)', u'Comedy|Romance')),
 (u'4', (u'Waiting to Exhale (1995)', u'Comedy|Drama|Romance')),
 (u'5', (u'Father of the Bride Part II (1995)', u'Comedy'))]

In [24]:
augmentedRatings = avgRatings01.join(movieInfo)

augmentedRatings.take(5)

[(u'74701', (3.09375, (u'Of Time and the City (2008)', u'Documentary'))),
 (u'110557', (3.2, (u'Graceland (2012)', u'Crime|Drama|Thriller'))),
 (u'121634', (3.0, (u'Open Heart (2013)', u'Documentary'))),
 (u'110001',
  (3.125,
   (u'Cosmic Psychos: Blokes You Can Trust (2013)', u'Documentary|Musical'))),
 (u'3480', (3.4320342205323193, (u'Lucas (1986)', u'Drama|Romance')))]

*Movie with highest average rating:*

In [25]:
augmentedRatings.takeOrdered(10, key = lambda x : -x[1][0])

[(u'134613', (5.0, (u'Summer (1976)', u'Comedy|Drama'))),
 (u'134346',
  (5.0, (u'Samay: When Time Strikes (2003)', u'(no genres listed)'))),
 (u'149456', (5.0, (u'Rainbow Eyes (2007)', u'Romance|Thriller'))),
 (u'147454', (5.0, (u"Aleksandr's Price (2013)", u'Drama|Mystery|Thriller'))),
 (u'116106',
  (5.0, (u'Death of a Nation - The Timor Conspiracy (1994)', u'Documentary'))),
 (u'139988', (5.0, (u'"Rimini', u' Rimini (1987)"'))),
 (u'141434', (5.0, (u'My Friend Victoria (2014)', u'Drama'))),
 (u'150766',
  (5.0,
   (u'One Track Heart: The Story of Krishna Das (2013)', u'Documentary'))),
 (u'141203', (5.0, (u'Aakrosh (1980)', u'Drama'))),
 (u'141064', (5.0, (u'Uomo e galantuomo (1975)', u'Comedy')))]

*Movie with lowest average rating:*

In [26]:
augmentedRatings.takeOrdered(10, key = lambda x : x[1][0])

[(u'87962', (0.5, (u'Dreamkiller (2010)', u'Thriller'))),
 (u'139555', (0.5, (u'Kiss the Abyss (2010)', u'Horror'))),
 (u'94337', (0.5, (u'"First Texan', u' The (1956)"'))),
 (u'140323', (0.5, (u'Memories (2013)', u'Crime|Thriller'))),
 (u'100203', (0.5, (u'Sundome (2007)', u'Comedy|Drama|Romance'))),
 (u'139233', (0.5, (u'Sons of Liberty (2013)', u'Action|Sci-Fi'))),
 (u'110800', (0.5, (u'"Second Man', u' The (O Defteros Andras) (2013)"'))),
 (u'111040',
  (0.5,
   (u'"3 Holiday Tails (Golden Christmas 2: The Second Tail',
    u' A) (2011)"'))),
 (u'90114', (0.5, (u'I Dream Too Much (1935)', u'Comedy|Musical|Romance'))),
 (u'146858',
  (0.5, (u'Il figlio pi\xf9 piccolo (2010)', u'(no genres listed)')))]

### Challenge

- Augment the mapping process of WordCount with a function to filter out punctuations and capitalization from the unique words

### Challenge:

1. Make appropriate changes so that only movies with averaged ratings higher than 3.75 are collected
2. Further enhance your modification so that only movies with averaged ratings higher than 3.75 and number of ratings of at least 1000 times are collected.

### 4.2 Find genres which have the highest average ratings over the years

- Identify the genres associated with a movie and its rating
- Each movie can have multiple genres. How to flip the Key/Value pair?

In [ ]:
movieRatings.take(5)

In [ ]:
movieInfo.take(5)

In [ ]:
augmentedInfo = movieRatings.join(movieInfo)

In [ ]:
augmentedInfo.take(5)

In [ ]:
def extractGenreRating (t):
    final_tuples = []
    genreList = t[1][1][1].split("|")
    for genre in genreList:
        final_tuples.append((genre,t[1][0]))
    return final_tuples

print(extractGenreRating((u'1', (3.0, (u'Toy Story (1995)', u'Adventure|Animation|Children|Comedy|Fantasy')))))

In [ ]:
genreRatings = augmentedInfo.flatMap(extractGenreRating)

In [ ]:
genreRatings.take(5)

### Challenge:

Complete the remaining portion of task 2.2: Calculating the average rating of each genre over the years

### 4.3 Find users who rate movies most frequently in order to contact them for in-depth marketing analysis

- How do you define "frequently"?
    - At least once per week?

In [ ]:
userRatings = ratingsOnly.map(lambda line: (line.split(",")[0], float(line.split(",")[3])))

In [ ]:
ratingGroupByUsers = userRatings.groupByKey().mapValues(list)
ratingGroupByUsers.take(5)

In [ ]:
avgRatingFreq = ratingGroupByUsers.mapValues(lambda V: (max(V) - min(V)) / float(len(V)))
avgRatingFreq.take(5)

In [ ]:
x = [1346139060.0,
   1346139098.0,
   1346139113.0,
   1346139053.0,
   1346139234.0,
   1346139006.0,
   1346139209.0,
   1346139147.0,
   1346138998.0,
   1346139206.0,
   1346139224.0,
   1346139174.0,
   1346139152.0,
   1346139230.0,
   1346139181.0,
   1346139159.0,
   1346139314.0]
(max(x) - min(x)) / float(len(x))

In [ ]:
topUsers = avgRatingFreq.top(10, key=lambda x: x[1])

In [ ]:
topUsers

## 5. Airlines

**Spark SQL**
- Spark module for structured data processing
- provide more information about the structure of both the data and the computation being performed for additional optimization
- execute SQL queries written using either a basic SQL syntax or HiveQL

**DataFrame**
- distributed collection of data organized into named columns
- conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood
- can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs.

In [3]:
sqlContext = SQLContext(sc)
sqlContext

In [4]:
airlines = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("intro-to-hadoop/airlines/data/")\
    .cache()

In [5]:
%%time
airlines.count()

CPU times: user 32.2 ms, sys: 8.07 ms, total: 40.3 ms
Wall time: 3min 17s


123534969

In [6]:
%%time
airlines.count()

CPU times: user 1.12 ms, sys: 278 µs, total: 1.4 ms
Wall time: 489 ms


123534969

In [7]:
airlines.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Carr

You can interact with a DataFrame via SQLContext using SQL statements by registerting the DataFrame as a table

In [8]:
airlines.registerTempTable("airlines")

*How many unique airlines are there?*

In [9]:
uniqueAirline = sqlContext.sql("SELECT DISTINCT UniqueCarrier \
                                FROM airlines")
uniqueAirline.show()

+-------------+
|UniqueCarrier|
+-------------+
|           AA|
|       PA (1)|
|           TW|
|           TZ|
|           AQ|
|           HA|
|           AS|
|           UA|
|           B6|
|           NW|
|           HP|
|           US|
|           OH|
|           OO|
|           PI|
|           CO|
|       ML (1)|
|           PS|
|           WN|
|           DH|
+-------------+
only showing top 20 rows



*Calculate how many flights completed by each carrier over time*

In [10]:
%%time
carrierFlightCount = sqlContext.sql("SELECT UniqueCarrier, COUNT(UniqueCarrier) AS FlightCount \
                                    FROM airlines GROUP BY UniqueCarrier")
carrierFlightCount.show()

+-------------+-----------+
|UniqueCarrier|FlightCount|
+-------------+-----------+
|           AA|   14984647|
|       PA (1)|     316167|
|           TW|    3757747|
|           TZ|     208420|
|           AQ|     154381|
|           HA|     274265|
|           AS|    2878021|
|           UA|   13299817|
|           B6|     811341|
|           NW|   10292627|
|           HP|    3636682|
|           US|   14075530|
|           OH|    1464176|
|           OO|    3090853|
|           PI|     873957|
|           CO|    8145788|
|       ML (1)|      70622|
|           PS|      83617|
|           WN|   15976022|
|           DH|     693047|
+-------------+-----------+
only showing top 20 rows

CPU times: user 2.03 ms, sys: 1.81 ms, total: 3.84 ms
Wall time: 2.98 s


*How do you display full carrier names?*

In [11]:
carriers = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("intro-to-hadoop/airlines/metadata/carriers.csv")\
    .cache()
carriers.registerTempTable("carriers")

In [12]:
carriers.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [13]:
%%time
carrierFlightCountFullName = sqlContext.sql("SELECT c.Description, a.UniqueCarrier, COUNT(a.UniqueCarrier) AS FlightCount \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.UniqueCarrier, c.Description \
                                    ORDER BY a.UniqueCarrier")
carrierFlightCountFullName.show()

+--------------------+-------------+-----------+
|         Description|UniqueCarrier|FlightCount|
+--------------------+-------------+-----------+
|Pinnacle Airlines...|           9E|     521059|
|American Airlines...|           AA|   14984647|
| Aloha Airlines Inc.|           AQ|     154381|
|Alaska Airlines Inc.|           AS|    2878021|
|     JetBlue Airways|           B6|     811341|
|Continental Air L...|           CO|    8145788|
|    Independence Air|           DH|     693047|
|Delta Air Lines Inc.|           DL|   16547870|
|Eastern Air Lines...|           EA|     919785|
|Atlantic Southeas...|           EV|    1697172|
|Frontier Airlines...|           F9|     336958|
|AirTran Airways C...|           FL|    1265138|
|Hawaiian Airlines...|           HA|     274265|
|America West Airl...|           HP|    3636682|
|Midway Airlines I...|       ML (1)|      70622|
|American Eagle Ai...|           MQ|    3954895|
|Northwest Airline...|           NW|   10292627|
|         Comair Inc

*What is the averaged departure delay time for each airline?*

In [ ]:
%%time
avgDepartureDelay = sqlContext.sql("SELECT c.Description, a.UniqueCarrier, AVG(a.DepDelay) AS AvgDepDelay \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.DepDelay, c.Description \
                                    ORDER BY a.UniqueCarrier")
carrierFlightCountFullName.show()

In [ ]:
airlines.unpersist()